<a href="https://colab.research.google.com/github/shahnupur1901/Filtrone-App/blob/master/DeepLearningModels/ModelForSubcategory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download paramaggarwal/fashion-product-images-small
! unzip /content/fashion-product-images-small.zip

In [ ]:
import numpy as np
import pandas as pd
import os
path = "/content/myntradataset"
csv_path = "/content/myntradataset/styles.csv"
img_path = "/content/myntradataset/images"
df = pd.read_csv(csv_path, error_bad_lines=False, warn_bad_lines=False)

df = df[df['baseColour'].notna()]
df['id'] = df['id'].apply(lambda x: str(x)+'.jpg')
df = df[df['id'].isin(os.listdir(img_path))]
df = df.reset_index(drop=True)
print('DF shape:', df.shape)

DF shape: (44404, 10)


In [ ]:
df = df.groupby(['subCategory']).filter(lambda x: len(x) >= 30)
df.shape[0]
df.sample(frac=1, axis=1)

,usage,id,baseColour,year,subCategory,masterCategory,articleType,season,gender,productDisplayName
0,Casual,15970.jpg,Navy Blue,2011.0,Topwear,Apparel,Shirts,Fall,Men,Turtle Check Men Navy Blue Shirt
1,Casual,39386.jpg,Blue,2012.0,Bottomwear,Apparel,Jeans,Summer,Men,Peter England Men Party Blue Jeans
2,Casual,59263.jpg,Silver,2016.0,Watches,Accessories,Watches,Winter,Women,Titan Women Silver Watch
3,Casual,21379.jpg,Black,2011.0,Bottomwear,Apparel,Track Pants,Fall,Men,Manchester United Men Solid Black Track Pants
4,Casual,53759.jpg,Grey,2012.0,Topwear,Apparel,Tshirts,Summer,Men,Puma Men Grey T-shirt
...,...,...,...,...,...,...,...,...,...,...
44399,Casual,17036.jpg,White,2013.0,Shoes,Footwear,Casual Shoes,Summer,Men,Gas Men Caddy Casual Shoe
44400,Casual,6461.jpg,Red,2011.0,Flip Flops,Footwear,Flip Flops,Summer,Men,Lotto Men's Soccer Track Flip Flop
44401,Casual,18842.jpg,Blue,2011.0,Topwear,Apparel,Tshirts,Fall,Men,Puma Men Graphic Stellar Blue Tshirt
44402,Casual,46694.jpg,Blue,2017.0,Fragrance,Personal Care,Perfume and Body Mist,Spring,Women,Rasasi Women Blue Lady Perfume


In [ ]:
df.groupby(['subCategory']).size()

subCategory
Apparel Set                   106
Bags                         3055
Belts                         811
Bottomwear                   2687
Dress                         478
Eyewear                      1073
Flip Flops                    913
Fragrance                    1003
Free Gifts                    104
Headwear                      293
Innerwear                    1808
Jewellery                    1079
Lips                          527
Loungewear and Nightwear      470
Mufflers                       38
Sandal                        963
Saree                         427
Shoes                        7343
Socks                         698
Ties                          258
Topwear                     15398
Wallets                       933
Watches                      2542
dtype: int64

In [ ]:
df = df[df.subCategory != "Accessories"]
df = df[df.subCategory != "Skin Care"]
df = df[df.subCategory != "Muffler"]
df = df[df.subCategory != "Scarves"]
df = df[df.subCategory != "Cufflinks"]
df = df[df.subCategory != "Eyes"]
df = df[df.subCategory != "Makeup"]
df = df[df.subCategory != "Nails"]
df = df[df.subCategory != "Stoles"]
df = df[df.subCategory != "Wallet"]

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df,test_size = 0.15, random_state=42)
train_df, validation_df = train_test_split(df,test_size = 0.15, random_state=57)

In [ ]:
df.head()

batch_size = 16

from keras_preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(
    rescale= 1./255,
    shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    #validation_split=0.2
)
validation_image_generator = ImageDataGenerator(
    rescale= 1./255,
    #validation_split=0.2
)

training_generator = train_image_generator.flow_from_dataframe(
    dataframe=train_df,
    directory=img_path,
    x_col="id",
    y_col="subCategory",
    target_size=(96,96),
    batch_size=batch_size,
    #subset="training"
)

validation_generator = validation_image_generator.flow_from_dataframe(
    dataframe=validation_df,
    directory=img_path,
    x_col="id",
    y_col="subCategory",
    target_size=(96,96),
    batch_size=batch_size,
    #subset="validation"
)
test_generator = validation_image_generator.flow_from_dataframe(
    dataframe=test_df,
    directory=img_path,
    x_col="id",
    y_col="subCategory",
    target_size=(96,96),
    batch_size=batch_size,
    #subset="validation"
)

classes = len(training_generator.class_indices)
#print(training_generator.class_indices)
#print(validation_generator.class_indices)

Found 36555 validated image filenames belonging to 23 classes.
Found 6452 validated image filenames belonging to 23 classes.
Found 6452 validated image filenames belonging to 23 classes.


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization,Dropout
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(32,(3,3), activation = 'relu', input_shape = (96,96,3)))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(32,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(classes, activation='softmax'))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005), loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])
model.summary()
batch_size = 16
total_train = train_df.shape[0]
total_validation = validation_df.shape[0]
vgg_classifier = model.fit(training_generator,
steps_per_epoch=(total_train//batch_size),
epochs = 10,
validation_data=validation_generator,
validation_steps=(total_validation//batch_size),
batch_size = batch_size,
verbose=1)

In [ ]:
model.save('modelForSubcategory.h5')
result = model.evaluate(test_generator,batch_size=16)
print("test_loss, test accuracy",result)

416/416 [==============================] - 19s 46ms/step - loss: 0.2287 - accuracy: 0.9303
test_loss, test accuracy [0.2286962866783142, 0.9302920699119568]
